# Project Euler problem 659 - Largest prime

[Link to problem on Project Euler homepage](https://projecteuler.net/problem=659)

## Description

Consider the sequence $n^2+3$ with $n \geq 1$.

If we write down the first terms of this sequence we get: $4,7,12,19,28,39,52,67,84,103,124,147,172,199,228,259,292,327,364,\ldots$.

We see that the terms for $n = 6$ and $n = 7$ (39 and 52) are both divisible by 13. In fact 13 is the largest prime dividing any two successive terms of this sequence.

Let $P(k)$ be the largest prime that divides any two successive terms of the sequence $n^2+k^2$.

Find the last 18 digits of $\sum_{k=1}^{10000000} P(k)$.

## Preliminary investigation
If the nth term of the sequence can be written as $n^2 + k^2$ then the subsequent term can be written as

$$(n+1)^2 + k^2 = n^2 + k^2 + 2n + 1.$$

If $n^2 + k^2$ can be divided by an integer, $p$, then $2n + 1$ must also be divisable by $p$, otherwise $(n+1)^2 + k^2$ cannot be divisable by $p$.

This means that

$$m = \frac{n^2 + k^2}{2n + 1}$$

will be an integer. Solving this for $n$ while assuming $k$ and $m$ positive yields

$$n = \sqrt{m^2 - k^2 + m} + m.$$

For $n$ to come out as an integer, the expression under the square root must be a square number. This is easily seen to be true when $m = k^2$, which yields $n = 2k^2$.

At this point I am going to conjecture that there are no other integer solutions to the equation. This means that we need to find the largest prime factor for each $4k^2 + 1$ for each value of $k$ between $1$ and $10^7$.

## Algorithm

The problem can be solved with [trial division](https://en.wikipedia.org/wiki/Trial_division). For each $k$, we loop over the primes $< \sqrt{4k^2 + 1}$ to see if they divide $4k^2 + 1$. If, at the end, what remains of $4k^2 + 1$ is a $1$ it means that the largests of the generated primes is the largest prime factor. If the remainder is $> 1$ it means that the remainder itself is the largest prime factor.

The last point is because if you have $p \times q = n$ where both $p$ and $q$ are primes, one of the factors must be $< \sqrt{n}$. If one does trial division of all primes up to $\sqrt{n}$, that means that if whatever remains is $> 1$ it must be prime.

As one last optimisation I noticed, empirically, that the only primes, $p$, that divide $4k^2 + 1$ are those where $(p - 1) \mod 4 = 0$. This optimisation is included into the algorithm

In [1]:
from time import time
from sympy.ntheory import primerange

def p659(kmax):
    t0 = time()

    primes = list(primerange(1, 2*kmax))
    primes = [prime for prime in primes if (prime - 1) % 4 == 0 ]

    total = 0
    for k in range(1, kmax+1):
        rem = 4*k*k + 1
        biggest_prime_divisor = 1

        for prime in primes:
            while prime*prime <= rem and rem % prime == 0:
                rem //= prime
                biggest_prime_divisor = prime
            if prime*prime > rem:
                break
        total += biggest_prime_divisor if rem == 1 else rem
    return total, time()-t0

for kmax in [10**3, 10**4, 10**5, 10**6]:
    total, seconds = p659(kmax)
    minutes, seconds = divmod(seconds, 60)
    hours, minutes = divmod(minutes, 60)

    print("")
    print("kmax = {}".format(kmax))
    print("total = {}".format(total))
    print("Time elapsed = {:.0f}h {:.0f}m {:f}s".format(hours, minutes, seconds))


kmax = 1000
total = 299015732
Time elapsed = 0h 0m 0.005808s

kmax = 10000
total = 223215627804
Time elapsed = 0h 0m 0.272908s

kmax = 100000
total = 178688812032788
Time elapsed = 0h 0m 19.521705s

kmax = 1000000
total = 148687122056813880
Time elapsed = 0h 21m 16.750166s


In pure Python it takes too long to run the algorithm for $`kmax` = 10^7$. Running the algorithm with [pypy](https://pypy.org) (which speeds up algorithm by roughly a factor of 20) it runs in 1h34m.